In [4]:
# !pip install tensorflow
# !pip install keras

In [2]:
import tensorflow as tf
print("TensorFlow version: ", tf.__version__)

2024-07-22 20:58:08.909373: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 20:58:08.923843: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-22 20:58:08.928191: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-22 20:58:08.939988: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-22 20:58:09.945291: W tensorflow/compiler/tf2

TensorFlow version:  2.17.0


In [6]:
import numpy as np

from keras.src.api_export import keras_export
from keras.src.utils.file_utils import get_file

@keras_export("keras.datasets.mnist.load_data")
def load_data(path="mnist.npz"):
    origin_folder = (
        "https://storage.googleapis.com/tensorflow/tf-keras-datasets/"
    )
    path = get_file(
        fname=path,
        origin=origin_folder + "mnist.npz",
        file_hash=(  # noqa: E501
            "731c5ac602752760c8e48fbffcf8c3b850d9dc2a2aedcf2cc48468fc17b673d1"
        ),
    )
    with np.load(path, allow_pickle=True) as f:
        x_train, y_train = f["x_train"], f["y_train"]
        x_test, y_test = f["x_test"], f["y_test"]

        return (x_train, y_train), (x_test, y_test)

In [9]:
# Load a dataset
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [25]:
# Build a machine learning model
# Build a tf.keras.Sequential model

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])
print(model)


<Sequential name=sequential_3, built=True>


/usr/local/python/3.10.13/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [24]:
# logits or log-odds scores
predictions = model(x_train[:1]).numpy()
print(predictions)

[[ 0.00124264 -0.01352623  0.3226814   0.72461134 -0.49262303  0.18841565
   0.38583186 -0.24519028  0.69582856  0.12892368]]


In [23]:
# tf.nn.softmax function
a = tf.nn.softmax(predictions).numpy()
print(a)

[[0.07916666 0.07800605 0.10917958 0.16319144 0.04831247 0.09546199
  0.11629669 0.06187538 0.15856129 0.08994841]]


In [26]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
print(loss_fn)

In [27]:
lossFunc_var = loss_fn(y_train[:1], predictions).numpy()
print(lossFunc_var)

2.349027


In [32]:
# setting the metrics parameter to accuracy
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])


In [54]:
# Train and evaluate your model
'''
    The epochs=5 parameter means that the model will go through the entire training dataset 5 times.
    I've used ephocs=10 to get better accuracy
'''

model.fit(x_train, y_train, epochs=10)

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9963 - loss: 0.0125
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9952 - loss: 0.0140
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9963 - loss: 0.0119
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9959 - loss: 0.0128
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9961 - loss: 0.0129
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9955 - loss: 0.0141
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9956 - loss: 0.0130
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9956 - loss: 0.0131
Epoch 9/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9957 - loss: 0.0119
Epoch 10/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9965 - loss: 0.0107


In [55]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - 717us/step - accuracy: 0.9811 - loss: 0.1240


[0.12395886331796646, 0.9811000227928162]

In [56]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [58]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.5411873e-21, 7.6662207e-29, 6.8330750e-17, 3.5146920e-12,
        1.9258119e-34, 2.8480569e-18, 0.0000000e+00, 1.0000000e+00,
        9.9114532e-20, 4.3295746e-15],
       [2.0892644e-31, 2.4358082e-21, 1.0000000e+00, 4.4025744e-22,
        0.0000000e+00, 1.5340147e-20, 1.0136938e-27, 0.0000000e+00,
        1.4437688e-21, 0.0000000e+00],
       [2.0487466e-22, 1.0000000e+00, 2.5428425e-12, 6.1076560e-16,
        1.7131790e-12, 5.4471167e-17, 5.0433433e-14, 5.5635971e-09,
        6.3236355e-10, 1.3486332e-22],
       [1.0000000e+00, 4.0130969e-25, 4.8129407e-11, 1.4329830e-19,
        5.4965944e-29, 3.1272077e-15, 1.2277006e-15, 2.7753777e-14,
        1.0986681e-27, 2.5466980e-18],
       [1.5451151e-19, 0.0000000e+00, 4.4498105e-17, 9.7232031e-26,
        9.9999988e-01, 2.9863989e-19, 3.1657717e-23, 1.5358190e-11,
        1.8946525e-20, 1.7568300e-07]], dtype=float32)>